In [1]:
from sklearn.datasets import load_digits
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA
import pickle
import copy

import pandas as pd
import seaborn as sns
import numpy
import torch
import scipy
import scipy.stats
import numpy as np

In [2]:
import os
dim=2
base_dir='/Users/phumon/Documents/Research/nlu-test-sets/params_mvirt'
file_name=f'alpha-lognormal-identity-dim18_theta-normal-identity_nosubsample_1.00_0.20'
exp_dir=os.path.join(base_dir, file_name)

print(exp_dir)


/Users/phumon/Documents/Research/nlu-test-sets/params_mvirt/alpha-lognormal-identity-dim18_theta-normal-identity_nosubsample_1.00_0.20


In [3]:
param_plot_stats = {}
keys = ['a', 'b', 'g', 't']
for key in keys:
    with open(os.path.join(exp_dir, 'plot_stats_pickles', f'{key}.p'), 'rb') as f:
        param_plot_stats[key] = pickle.load(f)

https://stackoverflow.com/questions/35407669/independent-component-analysis-ica-in-python

In [4]:
alpha_arr = np.vstack(param_plot_stats['a']['mean'])
pca = PCA(n_components=2, random_state=0)
new_alpha_ = pca.fit_transform(alpha_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [5]:
print(new_alpha_.shape)

(79033, 2)


In [8]:
from multi_virt_v2 import *

In [10]:
repo = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))

In [11]:
datasets="boolq,cb,commonsenseqa,copa,cosmosqa,hellaswag,rte,snli,wic,qamr,arct,mcscript,mctaco,mutual,mutual-plus,quoref,socialiqa,squad-v2,wsc,mnli,mrqa-nq,newsqa,abductive-nli,arc-easy,arc-challenge,piqa,quail,winogrande,anli"
data_names, responses, n_items = get_files(
    os.path.join(repo, 'data'),
    "csv",
    set(datasets.split(','))
)

In [12]:
task_metadata = pd.read_csv('../irt_scripts/task_metadata.csv')
task_metadata.set_index("jiant_name", inplace=True)
task_list = [x for x in task_metadata.index if x in data_names]

In [13]:
total = 0
task_name = []
task_format = []

for tname, size in zip(data_names, n_items):
    name = task_metadata.loc[tname]['taskname']
    total += size
    task_name += [name for _ in range(size)]
    task_format += [task_metadata.loc[tname]['format'] for _ in range(size)]
    
task_name = pd.DataFrame(task_name, columns=['task_name'])
task_format = pd.DataFrame(task_format, columns=['format'])
task_name_format = pd.concat([task_name, task_format], axis=1)

In [16]:
task_name_format

,task_name,format
0,AbductiveNLI,MC-sent
1,AbductiveNLI,MC-sent
2,AbductiveNLI,MC-sent
3,AbductiveNLI,MC-sent
4,AbductiveNLI,MC-sent
...,...,...
79028,WSC,MC-sent
79029,WSC,MC-sent
79030,WSC,MC-sent
79031,WSC,MC-sent


In [49]:

param = pd.concat([pd.DataFrame(new_alpha_.tolist()), task_name_format], axis=1)

In [50]:
param

,0,1,task_name,format
0,0.158559,-0.026714,AbductiveNLI,MC-sent
1,-0.147352,0.008106,AbductiveNLI,MC-sent
2,-0.080547,0.095921,AbductiveNLI,MC-sent
3,-0.083888,-0.003054,AbductiveNLI,MC-sent
4,-0.102934,-0.061754,AbductiveNLI,MC-sent
...,...,...,...,...
79028,0.090980,0.105743,WSC,MC-sent
79029,-0.139715,-0.034630,WSC,MC-sent
79030,0.082167,0.072593,WSC,MC-sent
79031,-0.013094,-0.011992,WSC,MC-sent


In [51]:
%matplotlib widget
import matplotlib.pyplot as plt

plt.figure()
plt.plot(param[0], param[1], "kx")
plt.show()




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [52]:
groups = param.groupby("task_name")
groups

In [53]:
plt.figure()

groups = param.groupby("format")
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=name)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

name:  MC-par
name:  MC-sent
name:  classification
name:  span selection


In [22]:
param[0]

0        0.158559
1       -0.147352
2       -0.080547
3       -0.083888
4       -0.102934
           ...   
79028    0.090980
79029   -0.139715
79030    0.082167
79031   -0.013094
79032    0.091564
Name: 0, Length: 79033, dtype: float64

# Theta

In [43]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
pca = PCA(n_components=2, random_state=0)
new_beta_ = pca.fit_transform(beta_arr)  # Reconstruct signals
#A_alpha_ = ica.mixing_  # Get estimated mixing matrix


In [44]:
params_beta = pd.concat([pd.DataFrame(new_beta_.tolist()), task_name_format], axis=1)

In [54]:
groups = params_beta.groupby("task_name")


In [55]:
plt.figure()
for name, group in groups:
    print("name: ",name)
    plt.plot(group[0], group[1], marker="o", linestyle="", label=name)
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

name:  ARC-C
name:  ARC-E
name:  ARCT
name:  AbductiveNLI
name:  BoolQ
name:  CB
name:  COPA
name:  CSQA
name:  CosmosQA
name:  HellaSwag
name:  MC-TACO
name:  MCScript
name:  MNLI
name:  MRQA-NQ
name:  MuTual
name:  MuTual+
name:  NewsQA
name:  PiQA
name:  QAMR
name:  QuAIL
name:  Quoref
name:  RTE
name:  SNLI
name:  SQuAD2.0
name:  SocialIQA
name:  WSC
name:  WiC
name:  Winogrande


In [ ]:
# END scatter plot

In [ ]:
alpha_arr

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

plt.figure()

models = [alpha_arr, S_alpha_]
names = ['Alpha (mixed signal)',
         'ICA recovered Alpha signals']
colors = ['red', 'steelblue', 'orange']

for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(2, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig) #, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()


In [ ]:
beta_arr = np.vstack(param_plot_stats['b']['mean'])
ica = FastICA(n_components=2, random_state=0)
S_beta_ = ica.fit_transform(beta_arr)  # Reconstruct signals
A_beta_ = ica.mixing_  # Get estimated mixing matrix


In [ ]:
models = [beta_arr, S_beta_]
names = ['Beta (mixed signal)',
         'ICA recovered Theta signals']
colors = ['red', 'steelblue', 'orange']

plt.figure()
for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(2, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.subplots_adjust(0.09, 0.04, 0.94, 0.94, 0.26, 0.46)
plt.show()

In [ ]:
A_alpha_.shape

In [ ]:
from sklearn.linear_model import Lasso
clf = Lasso(alpha=0.1)
clf.fit(S_alpha_, A_alpha_)